In [1]:
import pandas as pd

DATASETS_PATH = "../data/validation/results_analysis/"
DATASETS = ["AI_Act", "DSA", "GDPR"]

In [2]:
dfs = []
for dataset in DATASETS:
    file_path = DATASETS_PATH + f"obligation_filtering_{dataset}.xlsx"
    df = pd.read_excel(file_path)
    dfs.append(df)

len(dfs)

3

In [21]:
file_path = DATASETS_PATH + f"obligation_filtering_Overall.xlsx"
overall_df = pd.concat(dfs)
overall_df.to_excel(file_path, index=False)

dataset = "GDPR"
file_path = DATASETS_PATH + f"obligation_filtering_{dataset}.xlsx"
overall_df = pd.read_excel(file_path)

In [22]:
elements = [
    "Classification",
    "Justification"
]

In [23]:
import ast
import numpy as np

# Assuming final_results is a DataFrame
for element in elements:
    def safe_mean(value):
        """Convert string lists to actual lists, then compute mean."""
        if isinstance(value, str):
            try:
                value = ast.literal_eval(value)  # Convert string to list
            except (SyntaxError, ValueError):
                return np.nan  # Return NaN if conversion fails
        if isinstance(value, list):
            return np.mean(value)
        elif pd.notna(value):  # Convert single numbers to float
            return float(value)
        return np.nan  # Return NaN for missing values

    # Apply safe_mean to convert and compute mean
    value_mean = overall_df[element].apply(safe_mean)

    # Floor the mean and convert to integer
    overall_df[element] = value_mean.apply(lambda x: int(np.floor(x)) if pd.notna(x) else np.nan)

    # Create Agreement column
    overall_df[element + "_Agreement"] = value_mean.apply(lambda x: 1 if x in [0, 1] else 0)

overall_df.sample(n=10)

,ID,ReviewerPair,Classification,Justification,Classification_Agreement,Justification_Agreement
3,018.003.001,R1-R4,1,1,1,1
16,033.001.002,R2-R3,1,1,1,1
25,090.001.001,R2-R3,0,0,0,0
5,030.003.001,R1-R4,1,1,1,1
14,092.002.001,R1-R4,0,0,0,0
6,035.009.001,R1-R4,1,1,1,1
7,035.010.001,R1-R4,0,0,1,1
8,038.003.002,R1-R4,1,1,1,1
22,065.002.002,R2-R3,1,1,1,1
23,065.005.002,R2-R3,1,1,1,1


In [24]:
def calculate_metrics(column, target_df):
    """ Compute classification metrics for a given column (Classification or Justification). """
    correct_answers = (target_df[column] == 1).sum()  # Correct answers (1 means correct)
    wrong_answers = (target_df[column] == 0).sum()  # Wrong answers (0 means wrong)

    accuracy = correct_answers / (correct_answers + wrong_answers) if (correct_answers + wrong_answers) > 0 else 0

    return {
        "Accuracy": accuracy
    }

accuracy_results = {}
for element in elements:
    element_metrics = calculate_metrics(element, overall_df)
    # print(f"\n=== Metrics for {element} ===")

    accuracy_results[element] = element_metrics["Accuracy"]
    #
    # for key, value in element_metrics.items():
    #     print(f"{key}: {value:.4f}")
accuracy_results_agreement = {}
for element in elements:
    element_metrics = calculate_metrics(element + "_Agreement", overall_df)

    accuracy_results_agreement[element] = element_metrics["Accuracy"]
    # print(f"\n=== Agreement Metrics for {element} ===")
    # for key, value in element_metrics.items():
    #     print(f"{key}: {value:.4f}")

# print("Accuracy")
# print(json.dumps(accuracy_results, indent=4))
#
# print("Agreement")
# print(json.dumps(accuracy_results_agreement, indent=4))

# Create DataFrame
df = pd.DataFrame({
    "Lines": accuracy_results.keys(),
    "Accuracy": accuracy_results.values(),
    "Agreement": accuracy_results_agreement.values()
})

In [25]:
df.to_excel(f"../data/validation/results_analysis/obligation_filtering_{dataset}_Accuracy_Agreement.xlsx", index=False)